In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import geopandas as gpd # GeoPandas library for spatial analytics
from shapely.geometry import shape
from shapely.geometry import Point 

from urllib.parse import urlencode
import urllib.request, json 

import networkx as nx #library supporting networks

import matplotlib.pyplot as plt
import seaborn as sns # visualization styling package
%matplotlib inline 

In [2]:
#load airqulait data
air_quality = pd.read_csv("./Data/Air_Quality.csv")
air_quality

,Unique ID,Indicator ID,Name,Measure,Measure Info,Geo Type Name,Geo Join ID,Geo Place Name,Time Period,Start_Date,Data Value,Message
0,216498,386,Ozone (O3),Mean,ppb,CD,313,Coney Island (CD13),Summer 2013,06/01/2013,34.64,NaN
1,216499,386,Ozone (O3),Mean,ppb,CD,313,Coney Island (CD13),Summer 2014,06/01/2014,33.22,NaN
2,219969,386,Ozone (O3),Mean,ppb,Borough,1,Bronx,Summer 2013,06/01/2013,31.25,NaN
3,219970,386,Ozone (O3),Mean,ppb,Borough,1,Bronx,Summer 2014,06/01/2014,31.15,NaN
4,164876,383,Sulfur Dioxide (SO2),Mean,ppb,CD,211,Morris Park and Bronxdale (CD11),Winter 2008-09,12/01/2008,5.89,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
16117,671118,386,Ozone (O3),Mean,ppb,CD,306,Park Slope and Carroll Gardens (CD6),Summer 2020,06/01/2020,28.70,NaN
16118,671119,386,Ozone (O3),Mean,ppb,CD,305,East New York and Starrett City (CD5),Summer 2020,06/01/2020,29.56,NaN
16119,671120,386,Ozone (O3),Mean,ppb,CD,304,Bushwick (CD4),Summer 2020,06/01/2020,29.65,NaN
16120,671121,386,Ozone (O3),Mean,ppb,CD,303,Bedford Stuyvesant (CD3),Summer 2020,06/01/2020,29.28,NaN


In [6]:
air_quality["Name"].unique()

array(['Ozone (O3)', 'Sulfur Dioxide (SO2)', 'PM2.5-Attributable Deaths',
       'Boiler Emissions- Total SO2 Emissions',
       'Boiler Emissions- Total PM2.5 Emissions',
       'Boiler Emissions- Total NOx Emissions',
       'Air Toxics Concentrations- Average Benzene Concentrations',
       'Air Toxics Concentrations- Average Formaldehyde Concentrations',
       'PM2.5-Attributable Asthma Emergency Department Visits',
       'PM2.5-Attributable Respiratory Hospitalizations (Adults 20 Yrs and Older)',
       'PM2.5-Attributable Cardiovascular Hospitalizations (Adults 40 Yrs and Older)',
       'Traffic Density- Annual Vehicle Miles Traveled',
       'O3-Attributable Cardiac and Respiratory Deaths',
       'O3-Attributable Asthma Emergency Department Visits',
       'O3-Attributable Asthma Hospitalizations',
       'Traffic Density- Annual Vehicle Miles Traveled for Cars',
       'Traffic Density- Annual Vehicle Miles Traveled for Trucks',
       'Nitrogen Dioxide (NO2)', 'Fine Partic

In [7]:
air_quality["Name"].value_counts()

Fine Particulate Matter (PM2.5)                                                 5076
Nitrogen Dioxide (NO2)                                                          5076
Ozone (O3)                                                                      1692
Sulfur Dioxide (SO2)                                                            1128
PM2.5-Attributable Asthma Emergency Department Visits                            384
O3-Attributable Asthma Emergency Department Visits                               384
O3-Attributable Asthma Hospitalizations                                          384
Traffic Density- Annual Vehicle Miles Traveled for Cars                          214
Traffic Density- Annual Vehicle Miles Traveled                                   209
Traffic Density- Annual Vehicle Miles Traveled for Trucks                        209
PM2.5-Attributable Cardiovascular Hospitalizations (Adults 40 Yrs and Older)     192
O3-Attributable Cardiac and Respiratory Deaths                   

In [10]:
airptype = 'Fine Particulate Matter (PM2.5)'
air_quality_PM = air_quality.loc[air_quality['Name'] == airptype]
air_quality_PM.reset_index(inplace=True, drop=True)
air_quality_PM

,Unique ID,Indicator ID,Name,Measure,Measure Info,Geo Type Name,Geo Join ID,Geo Place Name,Time Period,Start_Date,Data Value,Message
0,168394,365,Fine Particulate Matter (PM2.5),Mean,mcg per cubic meter,CD,101,Financial District (CD1),Winter 2008-09,12/01/2008,15.74,NaN
1,168395,365,Fine Particulate Matter (PM2.5),Mean,mcg per cubic meter,CD,102,Greenwich Village and Soho (CD2),Winter 2008-09,12/01/2008,15.21,NaN
2,168396,365,Fine Particulate Matter (PM2.5),Mean,mcg per cubic meter,CD,103,Lower East Side and Chinatown (CD3),Winter 2008-09,12/01/2008,14.24,NaN
3,168397,365,Fine Particulate Matter (PM2.5),Mean,mcg per cubic meter,CD,104,Clinton and Chelsea (CD4),Winter 2008-09,12/01/2008,15.48,NaN
4,168398,365,Fine Particulate Matter (PM2.5),Mean,mcg per cubic meter,CD,105,Midtown (CD5),Winter 2008-09,12/01/2008,18.84,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
5071,669980,365,Fine Particulate Matter (PM2.5),Mean,mcg per cubic meter,CD,502,South Beach and Willowbrook (CD2),Winter 2019-20,12/01/2019,6.89,NaN
5072,669981,365,Fine Particulate Matter (PM2.5),Mean,mcg per cubic meter,CD,502,South Beach and Willowbrook (CD2),Summer 2020,06/01/2020,6.55,NaN
5073,669982,365,Fine Particulate Matter (PM2.5),Mean,mcg per cubic meter,CD,503,Tottenville and Great Kills (CD3),Annual Average 2020,01/01/2020,5.32,NaN
5074,669983,365,Fine Particulate Matter (PM2.5),Mean,mcg per cubic meter,CD,503,Tottenville and Great Kills (CD3),Winter 2019-20,12/01/2019,5.95,NaN
